In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sklearn
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import RandomForestClassifier
import warnings 
warnings.filterwarnings('ignore')

In [2]:
train=pd.read_csv(r"C:\Users\이재학\kakr\2019-2nd-ml-month-with-kakr\train.csv")
test=pd.read_csv(r"C:\Users\이재학\kakr\2019-2nd-ml-month-with-kakr\test.csv")
sample_submission=pd.read_csv(r"C:\Users\이재학\kakr\2019-2nd-ml-month-with-kakr\sample_submission.csv")

In [3]:
x=train.iloc[:,:]

In [4]:
t=[0 for i in range(len(x))]
for i in range(len(x)):
    t[i]=''.join(list(x.values[i][1])[0:8])
x['date']=t
for i in range(len(x)):
    x['date'][i]=int(x['date'][i])
x=x.sort_values(by=['date'],axis=0)
del x['date']
del x['id']
x=x.reset_index()
del x['index']

# 19960810T0000 ->날짜별로 arraysort 하기위해 date 형식 수정후에 오름차순 arraysort ->상관없어서 id번호랑 date지움->
# 새로 만들었더니 index이상하게 생겨서 index reset 하고 열 지움

In [5]:
q=[0 for i in range(len(x))]
for i in range(len(x)):
    q[i]=(x['sqft_living'][i]-x['sqft_living'].mean())/(x['sqft_living'].std())
    
r=[0 for i in range(len(x))]
for i in range(len(x)):
    r[i]=(x['sqft_lot'][i]-x['sqft_lot'].mean())/(x['sqft_lot'].std())

x['sqft_living']=q
x['sqft_lot']=r

a=x['condition']
a=(a-a.min())/(a.max()-a.min())
x['condition']=a

b=x['grade']
b=(b-b.min())/(b.max()-b.min())
x['grade']=b

m=[0 for i in range(len(x))]
for i in range(len(x)):
    m[i]=(x['sqft_above'][i]-x['sqft_above'].mean())/(x['sqft_above'].std())
n=[0 for i in range(len(x))]
for i in range(len(x)):
    n[i]=(x['sqft_basement'][i]-x['sqft_basement'].mean())/(x['sqft_basement'].std())
x['sqft_above']=m
x['sqft_basement']=n

h=[0 for i in range(len(x))]
for i in range(len(x)):
    h[i]=(x['sqft_living15'][i]-x['sqft_living15'].mean())/(x['sqft_living15'].std())
g=[0 for i in range(len(x))]
for i in range(len(x)):
    g[i]=(x['sqft_lot15'][i]-x['sqft_lot15'].mean())/(x['sqft_lot15'].std())
x['sqft_living15']=h
x['sqft_lot15']=g

waterfront 그대로 / view 그대로 /condition 정규화
grade -> 1~13 이고 2는 없음 / grade : King County grading 시스템 기준으로 매긴 집의 등급 
영향력있고 점점 올라감 // 이 feature 영향을 더주게 해야 할 듯 // 정규화 하자

+나머지 정규화 or 표준화

In [6]:
del x['yr_built']
del x['yr_renovated']
del x['zipcode']
del x['lat']
del x['long']      

# 상관없는 열 지우기

In [7]:
x_train=x.iloc[:,1:]
y_train=x.iloc[:,0:1]

# x_train / y_train 만들기

In [8]:
y=test.iloc[:,:]

In [9]:
del y['id']
del y['date']
del y['yr_built']
del y['yr_renovated']
del y['zipcode']
del y['lat']
del y['long']

In [10]:
a_1=[0 for i in range(len(y))]
for i in range(len(y)):
    a_1[i]=(y['sqft_living'][i]-y['sqft_living'].mean())/(y['sqft_living'].std())
    
a_2=[0 for i in range(len(y))]
for i in range(len(y)):
    a_2[i]=(y['sqft_lot'][i]-y['sqft_lot'].mean())/(y['sqft_lot'].std())

y['sqft_living']=a_1
y['sqft_lot']=a_2


a_3=y['condition']
a_3=(a_3-a_3.min())/(a_3.max()-a_3.min())
y['condition']=a_3

a_4=y['grade']
a_4=(a_4-a_4.min())/(a_4.max()-a_4.min())
y['grade']=a_4


a_5=[0 for i in range(len(y))]
for i in range(len(y)):
    a_5[i]=(y['sqft_above'][i]-y['sqft_above'].mean())/(y['sqft_above'].std())
a_6=[0 for i in range(len(y))]
for i in range(len(y)):
    a_6[i]=(y['sqft_basement'][i]-y['sqft_basement'].mean())/(y['sqft_basement'].std())
y['sqft_above']=a_5
y['sqft_basement']=a_6


a_7=[0 for i in range(len(y))]
for i in range(len(y)):
    a_7[i]=(y['sqft_living15'][i]-y['sqft_living15'].mean())/(y['sqft_living15'].std())
a_8=[0 for i in range(len(y))]
for i in range(len(y)):
    a_8[i]=(y['sqft_lot15'][i]-y['sqft_lot15'].mean())/(y['sqft_lot15'].std())
y['sqft_living15']=a_7
y['sqft_lot15']=a_8

In [11]:
y

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,sqft_living15,sqft_lot15
0,4,4.50,3.696215,2.090858,1.0,0,0,0.50,0.833333,2.558977,2.832320,3.985753,3.506413
1,3,2.50,-0.186159,-0.201951,2.0,0,0,0.50,0.500000,0.142907,-0.655534,0.590874,-0.186166
2,4,3.00,0.979653,-0.239456,2.0,0,3,0.50,0.666667,0.251630,1.555720,0.232764,-0.325871
3,2,1.00,-0.945037,-0.122856,1.0,0,0,0.75,0.500000,-0.690637,-0.655534,-1.314270,-0.283020
4,4,1.75,-0.483111,-0.239936,1.0,0,0,0.75,0.500000,-1.101369,1.076995,-0.827241,-0.287521
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6463,4,3.50,0.858672,-0.248183,3.0,0,0,0.50,0.583333,1.290540,-0.655534,-0.354536,-0.275820
6464,1,1.00,-1.835893,-0.311579,1.0,0,0,0.75,0.416667,-1.669146,-0.655534,-0.913187,-0.282825
6465,2,2.25,-0.879047,-0.334851,2.0,0,0,0.50,0.500000,-0.726878,-0.450366,-1.056431,-0.450744
6466,2,1.00,-1.281583,-0.322350,2.0,0,0,0.50,0.500000,-1.060296,-0.655534,-1.212567,-0.404450


In [12]:
x_test=y.iloc[0:6468,:]  

# 예측용 x_test 만들기 // 근데 sample은 6468갠데 test는 6556개임;

In [13]:
model=LinearRegression()
model.fit(x_train,y_train)
model.predict(x_test)
sample_submission['price']=model.predict(x_test)
sample_submission.to_csv('submission-linear.csv',index=False)


# 선형회귀

In [14]:
model=LogisticRegression()
model.fit(x_train,y_train)
model.predict(x_test)
sample_submission['price']=model.predict(x_test)
sample_submission.to_csv('submission-logistic.csv',index=False)

# 로지스틱회귀

In [15]:
k_fold=KFold(n_splits=10,shuffle=True,random_state=0)
clf=RandomForestClassifier(n_estimators=50,max_depth=6,random_state=0)
cross_val_score(clf,x_train,y_train,cv=k_fold,scoring='accuracy').mean()
clf.fit(x_train,y_train)
sample_submission['price']=clf.predict(x_test)
sample_submission.to_csv('submission-randomforest.csv',index=False)


# 랜덤포레스트